In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.ticker import FormatStrFormatter

In [2]:
import json
import os

In [3]:
# JSON file containing the results for this simulation run
results_file = "/Users/brianmccarthy/git_repos/results-analysis/configs/cv2x.json"

In [4]:
with open(results_file) as results_json:
    config = json.load(results_json)["cv2x"]

In [5]:
# Markers to use for this run
markers = [".", "o", "v", "^", "<", ">", "1", "2", "3", "4", "8", "s", "p", "P", "*", "h",
           "H", "+", "x", "X", "D", "d", "|", "_", 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [6]:
def create_bins(lower_bound, width, quantity):
    """ create_bins returns an equal-width (distance) partitioning.
        It returns an ascending list of tuples, representing the intervals.
        A tuple bins[i], i.e. (bins[i][0], bins[i][1])  with i > 0
        and i < quantity, satisfies the following conditions:
            (1) bins[i][0] + width == bins[i][1]
            (2) bins[i-1][0] + width == bins[i][0] and
                bins[i-1][1] + width == bins[i][1]
    """
    bins = []
    for low in range(lower_bound, lower_bound + quantity * width + 1, width):
        bins.append((low, low + width))
    return bins

In [7]:
def bin_fields(df, fields, bin_width=10, bin_quantity=49):
    """
    Bins multiple dfs into a single dictionary that can be used as an average for multiple fields across multiple
    runs
    :param df: dataframe to bin
    :param fields: fields to be binned.
    :param bin_width: width of each bin
    :param bin_quantity: total number of bins
    :return:
    """
    bins = create_bins(lower_bound=0, width=bin_width, quantity=bin_quantity)
    distances = []
    overall_fields = {}
    for interval in bins:
        upper_b = interval[1]
        distances.append(upper_b)

    for field in fields:
        overall_fields[field] = []

    overall_fields["distance"] = distances

    distance_col = config["results"]["distance"]

    for i in range(len(bins)):
        lower_b = bins[i][0]
        upper_b = bins[i][1]
        fields_temp = df[(df[distance_col] >= lower_b) & (df[distance_col] < upper_b)]
        for field in fields:
            if i < len(overall_fields[field]):
                overall_fields[field][i] = (fields_temp[field].mean() + overall_fields[field][i]) / 2
            else:
                overall_fields[field].append(fields_temp[field].mean())

    return overall_fields

In [8]:
def pdr_dist_individual(pdr, distances, label, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    ax.plot(distances, pdr, label=label, marker=markers[0], markevery=3)

    ax.set(xlabel='Distance (m)', ylabel='Packet Delivery Rate (PDR) %')
    ax.legend(loc='lower right')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 100])
    plt.yticks(np.arange(0, 101, step=10))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name))

In [9]:
def pdr_dist(pdrs, distances, labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(pdrs)):
        ax.plot(distances, pdrs[i], label=labels[i])

    ax.set(xlabel='Distance (m)', ylabel='Packet Delivery Rate (PDR) %')
    ax.legend(loc='lower left')
    ax.tick_params(direction='in')
    
    ax.set_ylim([0, 100])
    plt.yticks(np.arange(0, 101, step=10))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [10]:
def errors_dist_individual(distances, decoded, errors, error_labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    ax.plot(distances, decoded, label="Decoded", marker=self.markers[0], markevery=3)

    for i in range(len(errors)):
        ax.plot(distances, errors[i], label=error_labels[i], marker=self.markers[i+1], markevery=3)

    ax.legend(loc='center left')
    ax.set(xlabel='Distance (m)', ylabel='Messages Decoded')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()

    if store:
        fig.savefig("{}.png".format(plot_name))
    

In [11]:
def errors_dist(distances, decoded, decoded_labels, errors, error_labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(decoded)):
        ax.plot(distances, decoded[i], label=decoded_labels[i], marker=self.markers[i], markevery=3)

        for j in range(len(errors[i])):
            ax.plot(distances, errors[i][j], label=error_labels[i][j], marker=self.markers[i + j])

    ax.legend(loc='center left')
    ax.set(xlabel='Distance (m)', ylabel='Messages Decoded')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))
    
    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name))

## Import data and begin comparsion

In [14]:
artery_csv = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/base/artery.csv"
omnetpp_csv = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/base/omnetpp.csv"

In [15]:
a_df = pd.read_csv(artery_csv)
o_df = pd.read_csv(omnetpp_csv)

In [16]:
o_df.columns

Index(['EventNumber', 'Time', 'NodeID', 'cbr', 'grantStartTime', 'sciDecoded',
       'sciFailedHalfDuplex', 'sciNotDecoded', 'sciReceived', 'sciSent',
       'selectedNumSubchannels', 'selectedSubchannelIndex', 'senderID',
       'subchannelReceived', 'subchannelSent', 'subchannelsUsed',
       'subchannelsUsedToSend', 'tbDecoded', 'tbFailedButSCIReceived',
       'tbFailedDueToNoSCI', 'tbFailedHalfDuplex', 'tbReceived', 'tbSent',
       'txRxDistanceSCI', 'txRxDistanceTB', 'grantBreak'],
      dtype='object')

In [17]:
a_df.columns

Index(['EventNumber', 'Time', 'NodeID', 'cbr', 'grantRequests',
       'grantStartTime', 'sciDecoded', 'sciFailedHalfDuplex', 'sciNotDecoded',
       'sciReceived', 'sciSent', 'selectedNumSubchannels',
       'selectedSubchannelIndex', 'senderID', 'subchannelReceived',
       'subchannelSent', 'subchannelsUsed', 'subchannelsUsedToSend',
       'tbDecoded', 'tbFailedButSCIReceived', 'tbFailedDueToNoSCI',
       'tbFailedHalfDuplex', 'tbReceived', 'tbSent', 'txRxDistanceSCI',
       'txRxDistanceTB', 'grantBreak'],
      dtype='object')

In [18]:
# Number of vehicles comparison
print("Num vehicles in omnet++: {}".format(len(o_df.NodeID.unique())))
print("Difference in num vehicles: {}".format(len(o_df.NodeID.unique()) - len(a_df.NodeID.unique())))

Num vehicles in omnet++: 252
Difference in num vehicles: 0


In [19]:
# TB sent difference
o_df.tbSent.sum() - a_df.tbSent.sum()

127.0

In [20]:
# SCI sent difference
o_df.sciSent.sum() - a_df.sciSent.sum()

127.0

In [21]:
# Received vs non-received.
o_df.tbReceived.sum() - a_df.tbReceived.sum()

31133.0

In [22]:
# Scis received
o_df.sciReceived.sum() - a_df.sciReceived.sum()

31133.0

In [23]:
# More HD errors in OMNeT++ but realistically not significantly more.
o_df.sciFailedHalfDuplex.sum() - a_df.sciFailedHalfDuplex.sum()

861.0

In [24]:
# Scis not received
o_df.sciNotDecoded.sum() - a_df.sciNotDecoded.sum()

-379408.0

In [25]:
# TBs not received but SCI was
o_df.tbFailedButSCIReceived.sum() - a_df.tbFailedButSCIReceived.sum()

107604.0

In [26]:
# TBs received
o_df.tbDecoded.sum() - a_df.tbDecoded.sum()

302937.0